<a href="https://colab.research.google.com/github/emanuelebrizzi/bootcamp/blob/main/code_vulnerability_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
#from google.colab import files
#import zipfile
#import os

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Data upload**


In [ ]:
import os
import glob

# Directory path
drive_path = "/content/drive/MyDrive/bootcamp_file"

libpng_path = os.path.join(drive_path, "LibPNG")

libpng_vuln_path = os.path.join(libpng_path, "Vulnerable_functions")
libpng_non_vuln_path = os.path.join(libpng_path, "Non_vulnerable_functions")

vlc_path = os.path.join(drive_path, "VLC")

vlc_vuln_path = os.path.join(vlc_path, "Vulnerable_functions")
vlc_non_vuln_path = os.path.join(vlc_path, "Non_vulnerable_functions")

pidgin_path = os.path.join(drive_path, "Pidgin")

pidgin_vuln_path = os.path.join(pidgin_path, "Vulnerable_functions")
pidgin_non_vuln_path = os.path.join(pidgin_path, "Non_vulnerable_functions")

In [ ]:

def load_files_from_folder(folder, label):
    files = glob.glob(os.path.join(folder, "*.c"))
    data = []
    for file in files:
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            data.append((f.read(), label, os.path.basename(file)))
    return data


# **Replace Function names and Variable names**

In [ ]:
import re
import random
import string
from pathlib import Path

In [ ]:
# Removes comments from C code
def remove_comments(code):
    code = re.sub(r'//.*', '', code)
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    return code

# Extracts variable and function names from C code
def extract_identifiers(code):
    pattern = re.compile(r'\b([a-zA-Z_][a-zA-Z0-9_]*)\b')
    keywords = set(['int', 'char', 'float', 'double', 'return', 'if', 'else', 'while', 'for', 'do', 'switch', 'case', 'void'])
    identifiers = set(pattern.findall(code)) - keywords
    return identifiers

# Classifies identifiers as variables or functions
def categorize_identifiers(identifiers):
    var_counter, fun_counter = 1, 1
    replacements = {}
    for identifier in identifiers:
        if re.search(r'\b[A-Za-z_][A-Za-z0-9_]*\s*\(', identifier):
            replacements[identifier] = f'fun_{fun_counter}'
            fun_counter += 1
        else:
            replacements[identifier] = f'var_{var_counter}'
            var_counter += 1
    return replacements

# Replaces variable and function names with new names
def replace_identifiers(code, replacements):

    for old, new in replacements.items():
        code = re.sub(r'\b' + re.escape(old) + r'\b', new, code)
    return code

# Reads a C file, replaces names, and saves the new file
def process_c_file(filepath, new_filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        code = file.read()

    code_no_comments = remove_comments(code)
    identifiers = extract_identifiers(code_no_comments)
    replacements = categorize_identifiers(identifiers)
    new_code = replace_identifiers(code_no_comments, replacements)
    with open(new_filepath, 'w', encoding='utf-8') as file:
        file.write(new_code)

# Processes all C files in a directory
def process_directory(directory, new_directory_path):
    for filepath in Path(directory).glob("*.c"):
        new_filepath = os.path.join(new_directory_path, os.path.basename(filepath))
        process_c_file(filepath, new_filepath)

Cleanning code:

In [ ]:
import shutil

# Creation modified directory
modified_path = os.path.join(drive_path, "modified_files")
modified_vuln_path = os.path.join(modified_path, "Vulnerable_functions")
modified_non_vuln_path = os.path.join(modified_path, "Non_vulnerable_functions")

if os.path.exists(modified_vuln_path):
  shutil.rmtree(modified_vuln_path)
os.makedirs(modified_vuln_path, exist_ok=True)

if os.path.exists(modified_non_vuln_path):
  shutil.rmtree(modified_non_vuln_path)
os.makedirs(modified_non_vuln_path, exist_ok=True)


if os.path.exists(modified_vuln_path):
  shutil.rmtree(modified_vuln_path)
os.makedirs(modified_vuln_path, exist_ok=True)

if os.path.exists(modified_non_vuln_path):
  shutil.rmtree(modified_non_vuln_path)
os.makedirs(modified_non_vuln_path, exist_ok=True)

process_directory(libpng_vuln_path, modified_vuln_path)
process_directory(libpng_non_vuln_path, modified_non_vuln_path)


# **BLSTM**

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

# Load and label data
def load_data(directory, label):
    data = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            data.append((file.read(), label))
    return data

vulnerable_files = load_data(modified_vuln_path, label=1)
non_vulnerable_files = load_data(modified_non_vuln_path, label=0)

# Combine and shuffle dataset
dataset = vulnerable_files + non_vulnerable_files
np.random.shuffle(dataset)

texts, labels = zip(*dataset)

# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

# Pad sequences
max_length = 500  # Adjust based on dataset
data = pad_sequences(sequences, maxlen=max_length, padding='post')
labels = np.array(labels)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define BLSTM model
embedding_dim = 128
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model on training data
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Predict on test data
predictions = model.predict(X_test)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.7257 - loss: 0.5365 - val_accuracy: 0.9600 - val_loss: 0.1510
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9280 - loss: 0.3050 - val_accuracy: 0.9600 - val_loss: 0.1577
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9324 - loss: 0.2246 - val_accuracy: 0.9600 - val_loss: 0.1442
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9076 - loss: 0.2855 - val_accuracy: 0.9600 - val_loss: 0.1351
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9272 - loss: 0.2121 - val_accuracy: 0.9600 - val_loss: 0.1245
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9341 - loss: 0.1999 - val_accuracy: 0.9600 - val_loss: 0.1074
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9329 - loss: 0.1348 - val_accuracy: 0.9800 - val_loss: 0.1165
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9525 - loss: 0.1292 - val_accuracy: 0.9600 - v

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


In [ ]:
len(dataset)

622